In [12]:
from __future__ import division
import numpy as np
import pandas as pd
import os
from os import listdir
from IPython.display import display
import scipy.optimize as spo

def find_csv_filenames(path_to_dir, suffix=".csv"):
    filenames = os.listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith(suffix)]


def get_stock_names(data_folder):
    result_list = []

    for name in find_csv_filenames(os.getcwd() + data_folder):
        result_list.append(os.path.splitext(name)[0])

    return result_list

def get_data(data_folder, dates):
    df = pd.DataFrame(index=dates)

    stock_names = get_stock_names(data_folder)

    for name in stock_names:
        df_temp = pd.read_csv(os.getcwd() + data_folder + name + ".csv", index_col='Date', parse_dates=True, usecols=['Date', 'Adj Close'], na_values=['nan'])
        df_temp = df_temp.rename(columns={'Adj Close': name})
        df = df.join(df_temp)

        if name == 'SPY': # drop days SPY didn't trade
            df = df.dropna(subset=["SPY"])

    return df

def price_normalizer(df):
    return df / df.ix[0, :]

def set_portfolio_allocs(df, allocs):
    return df * allocs

def daily_rp(allocs, df):
#     result_df = pd.DataFrame(index=df.index)
    result_df = price_normalizer(df)
    result_df = set_portfolio_allocs(result_df, allocs)
    
    return (result_df.sum(axis=1) - 1)[1:]


def SR(allocs, df):
    mean = daily_rp(allocs, df).mean()
    std = daily_rp(allocs, df).std()
    
    return (mean / std)


def negSR(allocs, df):
     
    return - SR(allocs, df)


def SR_annual(SR, sampling_rate):
    
    return np.sqrt(sampling_rate) * SR

def fit_line(df, error_func):
    start_pos = np.ones(df.shape[1]) / df.shape[1]
    #Says one minus the sum of all variables must be zero
    cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})

    #Required to have non negative values
    bnds = tuple((0,1) for x in start_pos)

    result = spo.minimize(error_func, start_pos, args=(df, ), method='SLSQP', bounds=bnds ,constraints=cons)
    
    return result.x

dates = pd.date_range("2009-01-01", "2015-12-31")
# dates = pd.date_range("2009-01-01", "2009-01-31")
data = get_data("/stocksYahoo/", dates)
data_normalized = price_normalizer(data)

display(data_normalized)

display(daily_rp([0.4, 0.4, 0.1, 0.1], data_normalized))
display(daily_rp([0.4, 0.4, 0.1, 0.1], data_normalized).mean())
display(daily_rp([0.4, 0.4, 0.1, 0.1], data_normalized).std())

# display(SR([0.4, 0.4, 0.1, 0.1], data_normalized))
cf = fit_line(data_normalized, negSR)
print(cf)
print("Sharpe Ratio (daily): {}").format(SR(cf, data_normalized))
print("Sharpe Ratio (anualized): {}").format(SR_annual(SR(cf, data_normalized), 252))

# def daily_Rp(df, port_pm):
#     """This function takes a dataframe of daily prices of multiple stocks and their allocations,
#        and returns a dataframe of the daily returns of the porfolio for each day of the original dataframe.
#     """
    
    

,AAPL,GOOG,IBM,SPY
2009-01-02,1.000000,1.000000,1.000000,1.000000
2009-01-05,1.042204,1.020945,0.993705,0.998817
2009-01-06,1.025014,1.039649,1.021289,1.005486
2009-01-07,1.002865,1.002147,1.004807,0.975366
2009-01-08,1.021488,1.012044,0.997825,0.979346
2009-01-09,0.998127,0.980549,0.969440,0.958369
2009-01-12,0.976970,0.973142,0.981000,0.935349
2009-01-13,0.966501,0.978215,0.976765,0.937070
2009-01-14,0.940275,0.936667,0.952157,0.907595
2009-01-15,0.918788,0.930505,0.962802,0.907917


2009-01-05    0.024512
2009-01-06    0.028543
2009-01-07    0.000022
2009-01-08    0.011130
2009-01-09   -0.015749
2009-01-12   -0.028320
2009-01-13   -0.030730
2009-01-14   -0.063248
2009-01-15   -0.073211
2009-01-16   -0.075367
2009-01-20   -0.122829
2009-01-21   -0.062565
2009-01-22   -0.036876
2009-01-23   -0.014496
2009-01-26   -0.006859
2009-01-27    0.008401
2009-01-28    0.051789
2009-01-29    0.034141
2009-01-30    0.012693
2009-02-02    0.020222
2009-02-03    0.030718
2009-02-04    0.035220
2009-02-05    0.062245
2009-02-06    0.105934
2009-02-09    0.128467
2009-02-10    0.074241
2009-02-11    0.071906
2009-02-12    0.088903
2009-02-13    0.079349
2009-02-17    0.032788
                ...   
2015-11-18    5.168984
2015-11-19    5.215393
2015-11-20    5.281231
2015-11-23    5.227975
2015-11-24    5.246761
2015-11-25    5.217438
2015-11-27    5.216313
2015-11-30    5.213658
2015-12-01    5.247683
2015-12-02    5.196192
2015-12-03    5.131237
2015-12-04    5.300673
2015-12-07 

2.4780319428931272

1.4777749993525289

[  3.22683105e-16   8.94962057e-02   9.10503794e-01   0.00000000e+00]
Sharpe Ratio (daily): 2.37572689147
Sharpe Ratio (anualized): 37.7134952271
